### Final Testing

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

import tensorflow as tf

import pickle

In [4]:
# Import test datset
# Loading testing file from pickle file
test_file = open("CSV_files/Testing_file.pickle","rb")
x_test = pickle.load(test_file)
y_test = pickle.load(test_file) 
print("x_test:",x_test.shape,"y_test", y_test.shape)

x_test: (2000, 11) y_test (2000,)


## Importing a pre-trained model:

### a) Create the network:
You can create the network by writing python code to create each and every layer manually as the original model. However, if you think about it, we had saved the network in .meta file which we can use to recreate the network using tf.train.import() function like this:

In [7]:
saver = tf.train.import_meta_graph('ANN_model_files/my_test_model.meta')
type(saver)

tensorflow.python.training.saver.Saver

 import_meta_graph appends the network defined in .meta file to the current graph. So, this will create the graph/network for you but we still need to load the value of the parameters that we had trained on this graph.

### b) Load the parameters:
We can restore the parameters of the network by calling restore on this saver which is an instance of tf.train.Saver() class.

In [9]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('ANN_model_files/my_test_model.meta')
saver.restore(sess,tf.train.latest_checkpoint('ANN_model_files/'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ANN_model_files/my_test_model


In [10]:
# restore our model
with tf.Session() as sess:    
    #First let's load meta graph and restore weights
    saver = tf.train.import_meta_graph('ANN_model_files/my_test_model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('ANN_model_files/'))
    

INFO:tensorflow:Restoring parameters from ANN_model_files/my_test_model


In [1]:
from sklearn.metrics import classification_report

In [12]:
x = tf.placeholder('float', [None, x_test.shape[1]])
y = tf.placeholder('float')

def test_neural_network(x):
    # Here we call neural_network_model() function & passing variable x
    prediction = saver
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels = y) )
    # Add the optimizer
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    # launch the graph in a session
    with tf.Session() as sess:
        #These variables must be initialized before you can train a model.
        # it only add ops(AdamOptimizer) that will initialize the variables when run.
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(10):
                # Actually intialize the variables
                _, c = sess.run([optimizer, cost], feed_dict={x:x_test, y:y_test})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float')) *100
        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))
        return correct

# correct = test_neural_network(x)